# Generate the NonLinLoc config files
This notebook generate the necessary input file for NLL (config.in) using the output of the notebook 3_velocity_to_poly_nnl

Webase the input files on the two templates files:
- nlloc_runfile_template.in 

Based on the work of Loïs Papin

In [1]:
import os
import json

## set of input parameter
* poly_id: id of the polygon (int)

In [2]:
poly_id = 2
out_file = f'./NLL_config_files/nlloc_runfile_poly{poly_id}.in'
os.makedirs(f'./NLL_config_files/', exist_ok=True)
out_name = "delph"
# with open("cascadia_poly.json", 'r+') as file:
with open("../creating_polygons/cascadia_poly.json", 'r+') as file:
    file_data = json.load(file)
poly_param = file_data[poly_id-1]
poly_param

{'poly_id': 2.0,
 'olon': -121.3,
 'olat': 48.0,
 'plat1': 46.9,
 'plat2': 49.1,
 'xmin': -124.0,
 'xmax': 124.0,
 'xsep': 1.0,
 'ymin': -124.0,
 'ymax': 124.0,
 'ysep': 1.0,
 'zmin': -4.0,
 'zmax': 100.0,
 'zsep': 1.0,
 'minlon': -122.6,
 'maxlon': -120.0,
 'minlat': 46.9,
 'maxlat': 49.1}

## Generic control file statements
Set transform

In [3]:
def edit_trans_cmd(nll_cmd_lines, poly_param):
    # Find and replace the TRANS line in the .run file
    for i, line in enumerate(nll_cmd_lines):
        if line.startswith('TRANS'):
            nll_cmd_lines[i] = f"TRANS LAMBERT WGS-84 {poly_param['olat']} {poly_param['olon']} {poly_param['plat1']} {poly_param['plat2']} 0.0" + ' # modified' + '\n'
            return nll_cmd_lines

In [4]:
# Read the template file
with open('./template_NLL/nlloc_runfile_template.in', 'r') as file:
    nll_cmd_lines = file.readlines()

# modify TRANS
nll_cmd_lines = edit_trans_cmd(nll_cmd_lines, poly_param)

# Write the modified content back to a new file
with open(out_file, 'w') as file:
    file.writelines(line.rstrip() + '\n' for line in nll_cmd_lines)
print('TRANS lines have been modified successfully.')

TRANS lines have been modified successfully.


## Prepare config for event location
NLLoc performs earthquake locations in 3D models using non-linear search techniques.

Defining the commands:
* LOCFILES
* LOCGRID
* LOCSEARCH command may be needed.

In [5]:
def edit_locfiles_cmd(nll_cmd_lines, input_filename_obs, input_filename_ttgrid, output_filename_loc):
    # Find and replace the LOCFILES line in the .run file
    locfiles_command = 'LOCFILES ' + input_filename_obs + ' NLLOC_OBS ' + input_filename_ttgrid + ' ' + output_filename_loc
    for i, line in enumerate(nll_cmd_lines):
        if line.startswith('LOCFILES'):
            nll_cmd_lines[i] = locfiles_command + ' #modified' + '\n'
            return nll_cmd_lines
        
def edit_locgrid_cmd(nll_cmd_lines, poly_param):
    # Find and replace the VGGRID line in the .run file
    x_num = int(((poly_param['xmax'] - poly_param['xmin'])//poly_param['xsep'])+1)
    y_num = int(((poly_param['ymax'] - poly_param['ymin'])//poly_param['ysep'])+1)
    z_num = int(((poly_param['zmax'] - poly_param['zmin'])//poly_param['zsep'])+1)
    # Syntax 1: LOCGRID xNum yNum zNum xOrig yOrig zOrig dx dy dz gridType saveFlag
    locgrid_command = f"LOCGRID {x_num} {y_num} {z_num} {poly_param['xmin']} {poly_param['ymin']} {poly_param['zmin']}"\
                     f" {poly_param['xsep']} {poly_param['ysep']} {poly_param['zsep']} PROB_DENSITY  SAVE"
    for i, line in enumerate(nll_cmd_lines):
        if line.startswith('LOCGRID'):
            nll_cmd_lines[i] = locgrid_command + ' # modified' + '\n'
            return nll_cmd_lines

In [6]:
# Determine the filenames for LOCFILES
input_filename_obs = './nll_picks/NLL_picks.txt' # Picks
input_filename_ttgrid = f'../creating_polygons/time/{out_name}_poly{poly_id}_lcc' # Travel-time grid
output_filename_loc = './loc/test_loc' # Location of the events

# Read the modified .run file
with open(out_file, 'r') as file:
    nll_cmd_lines = file.readlines()

# Modify LOCFILES cmd
nll_cmd_lines = edit_locfiles_cmd(nll_cmd_lines, input_filename_obs, input_filename_ttgrid, output_filename_loc)

# Modify LOCGRID cmd
# now just using same grid as velocity
nll_cmd_lines = edit_locgrid_cmd(nll_cmd_lines, poly_param)

# Write the modified content back to a new file (or overwrite the original)
with open(out_file, 'w') as file:
    file.writelines(line.rstrip() + '\n' for line in nll_cmd_lines)

print('LOCFILES and LOCGRID line has been modified successfully.')

LOCFILES and LOCGRID line has been modified successfully.


In [7]:
# What does it look like after modifying the .run file for the commands
with open(out_file, 'r') as file:
    lines = file.readlines()
    non_comment_lines = [line.strip() for line in lines if not line.strip().startswith('#') and line.strip()]
    for line in non_comment_lines:
        print(line)

CONTROL 1 54321
TRANS LAMBERT WGS-84 48.0 -121.3 46.9 49.1 0.0 # modified
LOCSIG CRESCENT -- UO
LOCFILES ./nll_picks/NLL_picks.txt NLLOC_OBS ../creating_polygons/time/delph_poly2_lcc ./loc/test_loc #modified
LOCHYPOUT SAVE_NLLOC_ALL SAVE_HYPOINV_SUM FILENAME_PUBLIC_ID
LOCSEARCH OCT 17 11 6 0.001 100000 10000 0 0
LOCMETH EDT_OT_WT 9999.0 4 -1 -1 -1 -1 -1 1
LOCGAU 0.2 0.0
LOCGAU2 0.01 0.05 2.0
LOCPHASEID  P   P
LOCPHASEID  S   S
LOCQUAL2ERR 0.1 0.5 1.0 2.0 99999.9
LOCGRID 249 249 105 -124.0 -124.0 -4.0 1.0 1.0 1.0 PROB_DENSITY  SAVE # modified
LOCPHSTAT 9999.0 -1 9999.0 1.0 1.0 9999.9 -9999.9 9999.9
LOCANGLES ANGLES_YES 5
LOCMAG ML_HB 1.0 1.110 0.00189
